<center><img src="./imge/CLBLOGO.jpg" alt="创乐博" style="width: 300px;"/></center>

# 15.基于OpenCV4 二维码识别

@－－－－湖南创乐博智能科技有限公司－－－－<br>
@  文件名：15.基于OpenCV4 二维码识别.ipynb <br>
@  版本：V2.0 <br>
@  author: zhulin<br>
@  说明：基于OpenCV4 二维码识别<br>

In [2]:
# 载入必要的库
import cv2
import numpy as np
import sys
import time

In [3]:
# 线程函数操作库
import threading # 线程
import ctypes
import inspect

# 线程结束代码
def _async_raise(tid, exctype):
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        raise SystemError("PyThreadState_SetAsyncExc failed")
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

In [4]:
from jetcam.utils import bgr8_to_jpeg
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

QR_img = widgets.Image(format='jpeg', width=320, height=240)
testQRCode_img = widgets.Image(format='jpeg', width=320, height=240)

dispaly_img = widgets.HBox([testQRCode_img,QR_img])
display(dispaly_img)

In [5]:
dispW=320
dispH=240
flip=4
camSet='nvarguscamerasrc !  video/x-raw(memory:NVMM), width=3264, height=2464, format=NV12, framerate=21/1 ! nvvidconv flip-method='+str(flip)+' ! video/x-raw, width='+str(dispW)+', height='+str(dispH)+', format=BGRx ! videoconvert ! video/x-raw, format=BGR ! appsink'
cam= cv2.VideoCapture(camSet)

width=cam.get(cv2.CAP_PROP_FRAME_WIDTH)
height=cam.get(cv2.CAP_PROP_FRAME_HEIGHT)
print('width:',width,'height:',height)

width: 320.0 height: 240.0


## 载入图片QR二维码

In [1]:
import cv2
image = cv2.imread('./images/testQRCode.png')
image_qrCodeDetector = cv2.QRCodeDetector()
decodedText, points, _ = image_qrCodeDetector.detectAndDecode(image)
if points is not None:
    nrOfPoints = len(points)
    for i in range(nrOfPoints):
        nextPointIndex = (i+1) % nrOfPoints
        cv2.line(image, tuple(points[i][0]), tuple(points[nextPointIndex][0]), (255,0,0), 5)
    print(decodedText)
    testQRCode_img.value = bgr8_to_jpeg(image)
else:
    print("QR code not detected")

Makerobo


NameError: name 'bgr8_to_jpeg' is not defined

## 摄像头二维码实时识别

In [6]:
def Video_display():
    qrCodeDetector = cv2.QRCodeDetector()
    while True:
        ret, frame = cam.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
        t = time.time()
        data, points, _ = qrCodeDetector.detectAndDecode(gray)
        print("Time Taken for Detect and Decode : {:.3f} seconds".format(time.time() - t))     
        if points is not None:
            nrOfPoints = len(points)
            for i in range(nrOfPoints):
                nextPointIndex = (i+1) % nrOfPoints
                cv2.line(frame, tuple(points[i][0]), tuple(points[nextPointIndex][0]), (255,0,0), 5)
        QR_img.value = bgr8_to_jpeg(frame)
    cap.release()

In [7]:
# 开始线程
t = threading.Thread(target=Video_display)
t.setDaemon(True)
t.start()

Time Taken for Detect and Decode : 0.054 seconds
Time Taken for Detect and Decode : 0.045 seconds
Time Taken for Detect and Decode : 0.045 seconds
Time Taken for Detect and Decode : 0.046 seconds
Time Taken for Detect and Decode : 0.048 seconds
Time Taken for Detect and Decode : 0.053 seconds
Time Taken for Detect and Decode : 0.044 seconds
Time Taken for Detect and Decode : 0.054 seconds
Time Taken for Detect and Decode : 0.053 seconds
Time Taken for Detect and Decode : 0.044 seconds
Time Taken for Detect and Decode : 0.055 seconds
Time Taken for Detect and Decode : 0.043 seconds
Time Taken for Detect and Decode : 0.044 seconds
Time Taken for Detect and Decode : 0.045 seconds
Time Taken for Detect and Decode : 0.044 seconds
Time Taken for Detect and Decode : 0.051 seconds
Time Taken for Detect and Decode : 0.043 seconds
Time Taken for Detect and Decode : 0.044 seconds
Time Taken for Detect and Decode : 0.044 seconds
Time Taken for Detect and Decode : 0.045 seconds
Time Taken for Detec

In [9]:
# 结束线程
stop_thread(t)